In [1]:
import numpy as np
import tensorflow as tf

def make_val(train_data, val_size):

    val_idx = np.random.choice(range(len(train_data[0])), val_size, replace=False)
    val_data = (train_data[0][val_idx], train_data[1][val_idx])
    train_data = (np.delete(train_data[0], val_idx, 0), np.delete(train_data[1], val_idx, 0))

    return train_data, val_data

def make_onehot(dataset, cate_num):

    dataset = (dataset[0], np.squeeze(np.eye(cate_num)[dataset[1]]))

    return dataset

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_data, test_data = tf.keras.datasets.cifar10.load_data()

train_data, val_data = make_val(train_data, 2000)

train_data = make_onehot(train_data, 10) # Shape: ((48000, 32, 32, 3), (48000, 10))
val_data = make_onehot(val_data, 10) # Shape: ((2000, 32, 32, 3), (2000, 10))
test_data = make_onehot(test_data, 10) # Shape: ((10000, 32, 32, 3), (10000, 10))

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 10
BATCH_SIZE = 32
batch_size = tf.placeholder(tf.int64)
total_batch = len(train_data[0]) // BATCH_SIZE

In [4]:
# create the Dataset
X = tf.placeholder(tf.float32, [None, 32*32*3])
Y = tf.placeholder(tf.float32, [None, 10])

dataset = tf.data.Dataset.from_tensor_slices((X, Y)).shuffle(buffer_size=10000).batch(batch_size).repeat()

# create the iter
iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

In [5]:
# create the model
W1 = tf.Variable(tf.random_normal([32*32*3,10], stddev=0.01))
b1 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(features, W1) + b1

In [6]:
# define cost/loss & optimizer & accuracy
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
# train my model
for epoch in range(training_epochs):
    print('{} Epoch Start!'.format(epoch + 1))
    sess.run(iter.initializer, feed_dict={X: np.reshape(train_data[0],(-1,32*32*3)), Y: train_data[1], batch_size: BATCH_SIZE})
    total_cost = 0
    train_accu = 0

    for i in range(total_batch):
        a, c, _ = sess.run([accuracy, cost, optimizer])
        total_cost += c
        train_accu += a

    sess.run(iter.initializer, feed_dict={X: np.reshape(val_data[0],(-1,32*32*3)), Y: val_data[1], batch_size: len(val_data[0])})
    val_accu = sess.run(accuracy)

    print("Epoch: {}, Loss: {:.4f}, train Accu: {:.4f}, val Accu: {:.4f}".format(epoch + 1, total_cost / total_batch,
                                                                                 train_accu / total_batch, val_accu))
print('Learning Finished!')

1 Epoch Start!
Epoch: 1, Loss: 90.7215, train Accu: 0.2330, val Accu: 0.2400
2 Epoch Start!
Epoch: 2, Loss: 81.9838, train Accu: 0.2577, val Accu: 0.2225
3 Epoch Start!
Epoch: 3, Loss: 82.6375, train Accu: 0.2638, val Accu: 0.2010
4 Epoch Start!
Epoch: 4, Loss: 82.7504, train Accu: 0.2727, val Accu: 0.2425
5 Epoch Start!
Epoch: 5, Loss: 85.3138, train Accu: 0.2701, val Accu: 0.2825
6 Epoch Start!
Epoch: 6, Loss: 81.6012, train Accu: 0.2749, val Accu: 0.2750
7 Epoch Start!
Epoch: 7, Loss: 83.9155, train Accu: 0.2752, val Accu: 0.2465
8 Epoch Start!
Epoch: 8, Loss: 83.6345, train Accu: 0.2778, val Accu: 0.2545
9 Epoch Start!
Epoch: 9, Loss: 82.1389, train Accu: 0.2832, val Accu: 0.2635
10 Epoch Start!
Epoch: 10, Loss: 84.4009, train Accu: 0.2784, val Accu: 0.2780
Learning Finished!


In [ ]:
# Test model and check accuracy
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: np.reshape(test_data[0],(-1,32*32*3)), Y: test_data[1]}))